In [1]:
import json, random, subprocess
from datasets import load_dataset
from demo_class import FlexFlowDemo
import os

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_datasets(finetune_dataset_size=2, inference_file_path='inference_dataset.json', finetuning_file_path='finetuning_dataset.json'):
    """Creates the inference and finetuning datasets according to the data from https://huggingface.co/datasets/databricks/databricks-dolly-15k.
    Only the 'open_qa' and 'closed_qa' prompts without context are kept.
    The datasets are saved into the files given as arguments.

    Keyword arguments:
    dataset_size -- the number of prompts to consider
    inference_file_path -- the file in which to save the inference data
    finetuning_file_path -- the file in which to save the finetuning data
    """
    dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
    data = []
    for row in dataset:
        if len(data) == finetune_dataset_size:
            break
        if ("open_qa" in row['category'] or "closed_qa" in row['category']) and len(row['context']) == 0:
            data.append(row['instruction'] + " " + row['response'])
    with open(inference_file_path, 'w') as file:
        json.dump(data[:1], file)
    with open(finetuning_file_path, 'w') as file:
        json.dump(data, file, indent=2, separators=(',', ': '))

def download_models_used(base_model="meta-llama/Meta-Llama-3-8B", peft_model="goliaro/llama-3-8b-lora", refresh_cache=True):
    args = [peft_model, '--base_model_name', base_model]
    if refresh_cache:
        args.append('--refresh-cache')
    subprocess.run(['python', '../../utils/download_peft_model.py'] + args, capture_output=True, text=True)

In [3]:
configs_dict = {
    "num_gpus": 4,
    "memory_per_gpu": 14000,
    "zero_copy_memory_per_node": 40000,
    "num_cpus": 4,
    "legion_utility_processors": 4,
    "data_parallelism_degree": 1,
    "tensor_parallelism_degree": 1,
    "pipeline_parallelism_degree": 4,
    "offload": False,
    "offload_reserve_space_size": 8 * 1024,  # 8GB
    "use_4bit_quantization": False,
    "use_8bit_quantization": False,
    "enable_peft": True,
    "peft_activation_reserve_space_size": 1024,  # 1GB
    "peft_weight_reserve_space_size": 1024,  # 1GB
    "profiling": False,
    "inference_debugging": False,
    "fusion": False,
    "max_requests_per_batch": 1,
    "max_sequence_length": 256,
    "max_tokens_per_batch": 128,
    "max_training_steps": 4,
    "seed": 42,
}
model_configs = {
    "base_model": "meta-llama/Meta-Llama-3-8B",
    "inference_peft_model_id": "goliaro/llama-3-8b-lora",
    "finetuning_peft_model_id": "goliaro/llama-3-8b-lora",
    "cache_path": os.environ.get("FF_CACHE_PATH", ""),
    "refresh_cache": False,
    "full_precision": True,
    # relative paths
    "inference_dataset": "inference_dataset.json",
    "finetuning_dataset": "finetuning_dataset.json",
    "output_file": "peft_demo.txt",
}
generation_configs = {
    "do_sample": False,
    "temperature": 0.9,
    "topp": 0.8,
    "topk": 1,
}
finetuning_configs = {
    "learning_rate": 1.0,
    "momentum": 0.0,
    "weight_decay": 0.0,
    "nesterov": False,
}
# Merge dictionaries
configs_dict.update(model_configs)
configs_dict.update(generation_configs)
configs_dict.update(finetuning_configs)

In [4]:
random.seed(configs_dict["seed"])

create_datasets(inference_file_path=configs_dict["inference_dataset"], 
                finetuning_file_path=configs_dict["finetuning_dataset"])

download_models_used(base_model=configs_dict["base_model"],
                     peft_model=configs_dict["finetuning_peft_model_id"])

demo = FlexFlowDemo(configs_dict)

demo.initialize_flexflow()
demo.start_server()
demo.generate_finetuning()
demo.generate_inference()
demo.stop_server()

[0 - 7fec72884280]    0.300277 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7fec72884280]    0.300335 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7fec72884280]    0.300351 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7fec72884280]    0.300375 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7fec72884280]    0.300387 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7fec72884280]    0.300400 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7fec72884280]    0.300411 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7fec72884280]    0.300421 {3}{Mapper}: Enabled Control Replication Optimizations.
/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
workSpaceSize (128 M